In [2]:
# Preparation of the song metadata file, the track_id is encoded as numerical values to save space.

# import numpy as np
# #import pylab as Plot
# import pandas as pd
# import gc
# from sklearn.externals import joblib


# from sklearn.preprocessing import LabelEncoder

import numpy as np
import pylab as Plot
import pandas as pd
import gc
import os
#from sklearn.externals import joblib
import joblib #direct import working on my machine
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import hickle as hkl
import glob
from tqdm import tqdm


#data_path = 'D:/skip spotify code/track_features/'
#data_path = '/home/sc4/skip/track_features/'
# data_path = 'Skip_Data/'
data_path = 'C:/Users/13528/Task 1/Skip Data/'

song_fea_0 = pd.read_csv(data_path+'tf_000000000000.csv')
song_fea_1 = pd.read_csv(data_path+'tf_000000000001.csv')

song_fea_0 = pd.concat((song_fea_0,song_fea_1))

le = LabelEncoder()
#
song_fea_0['track_id'] = le.fit_transform(song_fea_0['track_id'])

joblib.dump(le, 'le_track_id.pkl')

song_fea_0['mode'] = le.fit_transform(song_fea_0['mode'])

song_fea_0.to_parquet('spotify_song_fea.parquet')

song_fea_1 = []
gc.collect()

0

In [6]:
# Conversion of the training session csv files to parquet files for smaller size and fast loading, the track_ids and session ids are encoded as numerical values to save space. The generated files are saved in the 'Skip_Data/' folder.
# If done correcly, the name of the generated parquet files should be something like "log_3_20180918_000000000000.csv.parquet"

import numpy as np
import pylab as Plot
import pandas as pd
import gc
import os

from sklearn.preprocessing import LabelEncoder
# from sklearn.externals import joblib
import joblib


le_track_id = joblib.load('le_track_id.pkl')

# data_path = 'Skip_Data/'
data_path = 'C:/Users/13528/Task 1/Skip Data/'

le = LabelEncoder()

# dirs = os.listdir( 'train_fold/' )
dirs = os.listdir('C:/Users/13528/Task 1/train_fold/')
count = 0
for json_file in dirs:    
    print(0, count/len(dirs))
    
#     train_data = pd.read_csv('train_fold/'+json_file)
    train_data = pd.read_csv('C:/Users/13528/Task 1/train_fold/'+json_file)
    
    le_session = LabelEncoder()

    train_data['session_id'] = le_session.fit_transform(train_data['session_id'])

    train_data['track_id_clean'] = le_track_id.transform(train_data['track_id_clean'])

    train_data.to_parquet(data_path+json_file+'.parquet')
     
    count = count + 1

0 0.0
0 0.1
0 0.2
0 0.3
0 0.4
0 0.5
0 0.6
0 0.7
0 0.8
0 0.9


In [7]:
# Conversion of the training session csv files to parquet files for smaller size and fast loading, the track_ids and session ids are encoded as numerical values to save space. The generated files are saved in the 'Skip_Data/' folder.
# There are two kinds of test files, namely files with names that start with "log_input" or "log_prehistory". If this part of code is executed correctly, the names of the generated parquet files should be correspondingly be something like "pred_20180718.parquet" and "prehist_20180810.parquet" for 


import hickle as hkl

# from sklearn.externals import joblib
import joblib
from sklearn.preprocessing import LabelEncoder
import glob
from tqdm import tqdm
import pandas as pd

le_track_id = joblib.load('le_track_id.pkl')

# test_files = glob.glob('test_fold/log_input_*_000000000000.csv')
test_files = glob.glob('C:/Users/13528/Task 1/test_fold/log_input_*_000000000000.csv')

for i in tqdm(range(len(test_files))):
    le = LabelEncoder()
    
    file = pd.read_csv(test_files[i])
    
#     prefix = 'test_fold/log_prehistory'
    prefix = 'C:/Users/13528/Task 1/test_fold/log_prehistory'
    
#     pre_file = pd.read_csv(prefix + test_files[i][33:])
    pre_file = pd.read_csv(prefix + test_files[i][41:])
    
    file['session_id'] = le.fit_transform(file['session_id'])
    
    pre_file['session_id'] = le.transform(pre_file['session_id'])
    
    pre_file['track_id_clean'] = le_track_id.transform(pre_file['track_id_clean'])
    file['track_id_clean'] = le_track_id.transform(file['track_id_clean'])
    
#     file.to_parquet('test_pred/'+ 'pred_'+test_files[i][34:42]+'.parquet')
    
#     pre_file.to_parquet('test_pred/'+ 'prehist_'+test_files[i][34:42]+'.parquet')
    file.to_parquet('C:/Users/13528/Task 1/test_pred/'+ 'pred_'+test_files[i][42:50]+'.parquet')
    
    pre_file.to_parquet('C:/Users/13528/Task 1/test_pred/'+ 'prehist_'+test_files[i][42:50]+'.parquet')
    

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.54s/it]


In [8]:
# Preparation of the file for Glove training.


import numpy as np
import pylab as Plot
import pandas as pd
import gc
import os
import glob

# data_path = 'Skip_Data/'
data_path = 'C:/Users/13528/Task 1/Skip Data/'

train_files = glob.glob(data_path + '*.csv.parquet')

from sklearn.preprocessing import LabelEncoder
# from sklearn.externals import joblib
import joblib

#data_path = 'D:/skip spotify code/track_features/'
#data_path = '/home/sc4/skip/track_features/'

le_track_id = joblib.load('le_track_id.pkl')

le = LabelEncoder()

#dirs = dirs[0:200]

all_data = []

count = 0
for file in train_files:    
    print(0, count/len(train_files))
    
    train_data = pd.read_parquet(file)
    
    cols = ['session_id', 'session_position', 'session_length', 'track_id_clean']
    
    train_data = train_data[cols]
    
    train_data['track_id_clean'] = train_data['track_id_clean'] + 1
    
    raw_data = np.array(train_data.values)*1

    raw_data = raw_data.astype(np.int)    
    
    n_session = np.max(train_data['session_id'])+1
    
    gc.collect()
    
    data = np.zeros((n_session*30))
    
    data[raw_data[:,0].astype(np.int)*30+raw_data[:,1].astype(np.int)-1] = raw_data[:,3]

    if count == 0:
        all_data = data
    else:
        all_data = np.concatenate((all_data,data))
     
    count = count + 1

np.savetxt('glove_data.txt',all_data,newline=' ', delimiter = ' ', fmt='%i') 

0 0.0


C:\Users\13528\AppData\Local\Temp/ipykernel_18508/521039522.py:45: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  raw_data = raw_data.astype(np.int)
C:\Users\13528\AppData\Local\Temp/ipykernel_18508/521039522.py:53: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note 

0 0.1
0 0.2
0 0.3
0 0.4
0 0.5
0 0.6
0 0.7
0 0.8
0 0.9


In [10]:
len(all_data)

58950990

In [27]:
# After learning the Glove embedding, let the embedding txt file be named as 'vectors_150.txt', this part of code convert the txt file to numpy format.

import numpy as np
import pylab as Plot
import pandas as pd
import gc
import hickle as hkl
#from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def get_coefs(word,*arr): return (word), np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open('vectors_150.txt'))

song_embedding_matrix = np.zeros((3706389,150))
# song_embedding_matrix = np.zeros((393006,150))

keys = embeddings_index.keys()

for i in range(0,song_embedding_matrix.shape[0]):
        tmp = embeddings_index.get(str(i))
        if tmp is not None:
            song_embedding_matrix[i,:] = tmp
            
# hkl.dump(song_embedding_matrix, 'song_embedding_matrix_150.hkl', mode='w', compression='gzip')
hkl.dump(song_embedding_matrix, 'song_embedding_matrix_150.hkl', mode='w')

In [28]:
joblib.dump(song_embedding_matrix, 'song_embedding_matrix_150.pkl')

['song_embedding_matrix_150.pkl']

In [29]:
song_embedding_matrix.shape

(3706389, 150)

In [30]:
# spotify_song_array_glove = hkl.load('song_embedding_matrix_150.hkl')
spotify_song_array_glove = joblib.load('song_embedding_matrix_150.pkl')
# os.path.getsize('song_embedding_matrix_150.hkl')

In [31]:
spotify_song_array_glove

array([[-0.30616799, -0.61709601, -0.22131801, ..., -0.30539101,
         1.09236097,  1.19929302],
       [ 0.30116001, -0.171077  , -0.110602  , ...,  0.26418701,
        -0.17385501, -0.245979  ],
       [-0.92715001, -0.61823601, -0.028583  , ..., -0.61596602,
        -0.52215099,  0.49527901],
       ...,
       [-0.14865901,  0.071279  , -0.24946199, ...,  0.026126  ,
        -0.008469  ,  0.036255  ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.22290801,  0.156873  , -0.325515  , ..., -0.051759  ,
        -0.086463  , -0.54713899]])

In [20]:
embeddings_index.get('0')

array([-0.306168, -0.617096, -0.221318,  0.620467,  0.251854, -0.269983,
       -0.814901,  0.220307,  0.232154, -0.834252,  0.046133,  0.463726,
       -1.259767,  0.260635, -0.068549, -0.817818, -1.242769,  0.193992,
       -0.022523,  0.139119, -0.809364, -0.015667, -0.024639,  0.226955,
       -0.223547, -0.475842, -0.34594 , -0.319357, -0.12704 ,  0.121938,
       -0.028495, -0.949348,  0.266388, -0.547475,  0.094223, -0.419741,
       -0.989508,  0.190096, -0.488746, -0.256873, -0.275853,  0.725631,
        4.279365, -0.123002, -0.129986, -1.528008,  0.531471, -0.029998,
        0.432335,  0.702673,  1.26482 , -0.477329,  0.323498, -0.343377,
       -0.244294,  0.680409,  0.684785, -0.297188,  0.211914, -0.342224,
        0.023286, -0.433466,  1.286423, -0.709696,  0.125279, -0.594115,
        0.639779,  0.365565,  0.405594, -0.081768, -0.186266,  0.054931,
       -0.678648,  1.156425, -1.190904, -0.108001,  0.302235,  0.594203,
       -0.05948 ,  0.036465, -0.922106,  0.210533, 

In [22]:
len(song_embedding_matrix[0])

150